In [ ]:
import numpy as np
import pandas as pd

pd.options.plotting.backend = "plotly"

In [ ]:
parameters = {
    "contact_rate": 1.,
    "recovery_rate": 0.333,
    "death_rate": 0.05,
    "population": 1000.,
    "seed": 10.,
    "start_time": 0.,
    "end_time": 20.,
}

## Checking how the model works inside

This section presents some code that shows an approximation of what is happening inside the model we just built and ran.
This is a sanity check that the outputs look as we would expect if we had coded this manually,
and is intended to provide some insight into what is going on under the surface.

In the example code below we use the [Euler method](https://en.wikipedia.org/wiki/Euler_method) to solve an ordinary differential equation (ODE) which is defined by the model's flows. We don't typically use Euler in _summer_ though, you can read more about the actual ODE solvers available to evaluate models [here](http://summerepi.com/examples/4-ode-solvers.html).

In [ ]:
TIMESTEP = 0.1
START_TIME = 0
END_TIME = 20

# Get times
time_period = END_TIME - START_TIME + 1
num_steps = time_period / TIMESTEP
times = np.linspace(START_TIME, END_TIME, num=int(num_steps))

# Define initial conditions and prepare for outputs
initial_conditions = np.array(
    [
        parameters["population"] - parameters["seed"], 
        parameters["seed"],
        0.
    ]
)
outputs = np.zeros((int(num_steps), 3))
outputs[0] = initial_conditions

# Model parameters
contact_rate = parameters["contact_rate"]
recovery_rate = parameters["recovery_rate"]
death_rate = parameters["death_rate"]

# Calculate outputs for each timestep
for t_idx, t in enumerate(times):
    if t_idx == 0:
        continue

    # Get some useful quantities that we'll need later
    flow_rates = np.zeros(3)
    compartment_sizes = outputs[t_idx - 1]
    num_sus = compartment_sizes[0]
    num_inf = compartment_sizes[1]
    num_pop = compartment_sizes.sum()
    
    # Apply the infection process
    force_of_infection = contact_rate * num_inf / num_pop  # Frequency-dependent
    infection_flow_rate = force_of_infection * num_sus
    flow_rates[0] -= infection_flow_rate
    flow_rates[1] += infection_flow_rate

    # Infectious take some time to recover
    num_inf = compartment_sizes[1]
    recovery_flow_rate = recovery_rate * num_inf
    flow_rates[1] -= recovery_flow_rate
    flow_rates[2] += recovery_flow_rate
    
    # Add an infection-specific death flow to the I compartment
    num_inf = compartment_sizes[1]
    death_flow_rate = num_inf * death_rate
    flow_rates[1] -= death_flow_rate
    
    # Calculate compartment sizes at next timestep given flowrates
    outputs[t_idx] = compartment_sizes + flow_rates * TIMESTEP  
    
compartments = (
    "susceptible",
    "infectious",
    "recovered",
)
outputs_df = pd.DataFrame(outputs, columns=compartments)
outputs_df.plot()